In [ ]:
import numpy as np

from proseflow.aws_utils import *
from proseflow.load import *
from proseflow.text import text_to_vector, create_index, query_index
from proseflow.roam_utils import roam_graph_to_blocks, roam_blocks_to_embeddings_index
from dotenv import load_dotenv
from IPython.display import JSON

In [ ]:
load_dotenv()

True

In [ ]:
BUCKET = "roam-export"

In [ ]:
roam_graph = read_json_from_s3(bucket=BUCKET, key="scify.json").get("data")[:50]

In [ ]:
# JSON(roam_graph)

In [ ]:
embedder = load("distilbert-base-nli-mean-tokens", input_type="SENTENCE_TRANSFORMER")

In [ ]:
roam_blocks = roam_graph_to_blocks(roam_graph)

In [ ]:
# JSON(roam_blocks)

In [ ]:
vectors = text_to_vector(list(roam_blocks.values()), embedder)

TypeError: 'convert_to_tensor' is an invalid keyword argument for encode()

In [ ]:
index_list = roam_blocks_to_embeddings_index(roam_blocks, vectors)

NameError: name 'vectors' is not defined

In [ ]:
retrieved_vectors = np.array([obj.get("embedding") for obj in index_list], dtype=np.float32)
retrieved_sentences = [obj.get("sentence") for obj in index_list]

In [ ]:
index = create_index(retrieved_vectors)

In [ ]:
query_index("Roam sucks", embedder, retrieved_sentences, index)

['They are both based on the faulty idea of [Transmissionism](((ToiqIqjZh))).',
 'experiments with organoid models are shit. Cellular models are irrepresentative',
 'Is it listed in a list? (not main resource focus)',
 '**Books and lectures are bad vehicles for knowledge transfer.** ',
 'In {{alias: ((844_cS3uq)) t-SNE}} global structure may be lost in favor of preserving local distances.',
 "[Explorable Explanations are Bespoke One-Offs and their Representations Don't Compose and so the Field is Not Accreting](https://twitter.com/andy_matuschak/status/1305264249032171520)",
 'Stylistic and things like "summary on top", **bold if main point**, **nesting** and ^^declarative naming^^',
 'When x, I want to x, so I can x',
 "^^Transmissionism^^: the assumption that knowledge is transferred straight from the sender's output to the receivers input",
 'avg researcher knows they will never be able to keep up with the literature']